In [1]:
import tensorflow as tf
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
directory = directory = ('/Users/trent/Downloads/resources/')
folders = os.listdir(directory)
folders.remove('IDC_regular_ps50_idx5')
folders.remove('.DS_Store')
test_folders = []
train_folders = []
count = 0
flag = 0
for folder in folders:
    class_zero = os.listdir(directory+folder+'/0/')
    count += len(class_zero)
    class_one = os.listdir(directory+folder+'/1/')
    count += len(class_one)
    if count > 222000:
        flag = 1
    if flag:
        test_folders.append(folder)
    else:
        train_folders.append(folder) 
print(count)

277524


In [3]:
train_image_paths = []
test_image_paths = []
for folder_name in train_folders:
    class_zero = os.listdir(directory+folder_name+'/0/')
    class_one = os.listdir(directory+folder_name+'/1/')
    if '.DS_Store' in class_zero:
        class_zero.remove('.DS_Store')
    if '.DS_Store' in class_one:
        class_one.remove('.DS_Store')
    for path in class_zero:
        train_image_paths.append(directory+folder_name+'/0/'+path)
    for path in class_one:
        train_image_paths.append(directory+folder_name+'/1/'+path)
for folder_name in test_folders:
    class_zero = os.listdir(directory+folder_name+'/0/')
    class_one = os.listdir(directory+folder_name+'/1/')
    if '.DS_Store' in class_zero:
        class_zero.remove('.DS_Store')
    if '.DS_Store' in class_one:
        class_one.remove('.DS_Store')
    for path in class_zero:
        test_image_paths.append(directory+folder_name+'/0/'+path)
    for path in class_one:
        test_image_paths.append(directory+folder_name+'/1/'+path)

In [4]:
X_train = []
y_train = []
X_test = []
y_test = []
class_zero_count = 0
class_one_count = 0
general_badness_count = 0
removed = 0
image_path=test_image_paths[0]
image = tf.keras.preprocessing.image.load_img(image_path)
for path in train_image_paths:
    class_number = 2
    image = tf.keras.preprocessing.image.load_img(path)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    if(image_arr.shape == (50,50,3)):
        if path.endswith('class0.png'):
            class_number = 0
            class_zero_count += 1
        elif path.endswith('class1.png'):
            class_number = 1
            class_one_count += 1
        else:
            class_number = 2
            general_badness_count += 1
        X_train.append(image_arr)
        y_train.append(class_number)
    else:
        removed += 1
for path in test_image_paths:
    class_number = 2
    image = tf.keras.preprocessing.image.load_img(path)
    image_arr = tf.keras.preprocessing.image.img_to_array(image)
    if(image_arr.shape == (50,50,3)):
        if path.endswith('class0.png'):
            class_number = 0
            class_zero_count += 1
        elif path.endswith('class1.png'):
            class_number = 1
            class_one_count += 1
        else:
            class_number = 2
            general_badness_count += 1
        X_test.append(image_arr)
        y_test.append(class_number)
    else:
        removed +=1
print(f'zero: {class_zero_count}')
print(f'one: {class_one_count}')      
print(f'bad: {general_badness_count}')
print(f'removed due to shape: {removed}')

zero: 196454
one: 78768
bad: 0
removed due to shape: 2302


In [5]:
model = tf.keras.models.Sequential()
#starting with 32 filters and (3,3) kernel size
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(2))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [7]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Epoch 1/10
6831/6831 [==============================] - 469s 69ms/step - loss: 7.6486 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 2/10
6831/6831 [==============================] - 398s 58ms/step - loss: 7.6486 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 3/10
6831/6831 [==============================] - 318s 47ms/step - loss: 7.6486 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 4/10
6831/6831 [==============================] - 796s 117ms/step - loss: 7.6485 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 5/10
6831/6831 [==============================] - 413s 60ms/step - loss: 7.6486 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 6/10
6831/6831 [==============================] - 384s 56ms/step - loss: 7.6485 - accuracy: 0.7281 - val_loss: 7.6547 - val_accuracy: 0.6587
Epoch 7/10
6831/6831 [==============================] - 335s 49ms/step - loss: 7.6486 - accuracy: 0.7281 - val_loss: 

In [10]:
model_one = tf.keras.models.Sequential()
#starting with 32 filters and (3,3) kernel size
model_one.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model_one.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_one.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_one.add(tf.keras.layers.MaxPooling2D((2, 2)))
model_one.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model_one.add(tf.keras.layers.Flatten())
model_one.add(tf.keras.layers.Dense(64, activation='relu'))
model_one.add(tf.keras.layers.Dense(32, activation='relu'))
model_one.add(tf.keras.layers.Dense(8, activation='relu'))
model_one.add(tf.keras.layers.Dense(2))

In [9]:
model.save('saved_model.h5')

In [11]:
model_one.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [12]:
model_one.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_one.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test))

Epoch 1/10
6831/6831 [==============================] - 402s 59ms/step - loss: 11.0377 - accuracy: 0.2808 - val_loss: 10.0447 - val_accuracy: 0.3413
Epoch 2/10
6831/6831 [==============================] - 422s 62ms/step - loss: 11.1030 - accuracy: 0.2719 - val_loss: 10.0447 - val_accuracy: 0.3413
Epoch 3/10
6831/6831 [==============================] - 463s 68ms/step - loss: 11.1030 - accuracy: 0.2719 - val_loss: 10.0447 - val_accuracy: 0.3413
Epoch 4/10
6831/6831 [==============================] - 420s 62ms/step - loss: 11.1030 - accuracy: 0.2719 - val_loss: 10.0447 - val_accuracy: 0.3413
Epoch 5/10
6831/6831 [==============================] - 430s 63ms/step - loss: 11.1030 - accuracy: 0.2719 - val_loss: 10.0447 - val_accuracy: 0.3413
Epoch 6/10
2721/6831 [==========>...................] - ETA: 4:18 - loss: 11.0928 - accuracy: 0.2726

KeyboardInterrupt: 